In [1]:
import os
from openai import OpenAI
import openai
from tool import videoreader
# 创建一个OpenAI客户端实例

In [2]:
dimension = 'imaging_quality'
from tool.framework import critic_agent
agent1 = critic_agent()
client = agent1.client

In [3]:
import json
data_prepath = r'D:\Astudying\VideoEval\data4dimensions'
# data_prepath = "../../data4dimensions/"
with open("./Human_anno/{}.json".format(dimension)) as f:
    human_anno = json.load(f)

eval_path ="./GPT4o_eval_results/{}/{}_llmeval_7inputsfewshot_3.json".format(dimension,dimension)
with open(eval_path,'r') as f:
    eval_results = json.load(f)

    
batch_stpath = '../batch_api/{}'.format(dimension)
if not os.path.exists(batch_stpath):
    os.makedirs(batch_stpath)

batch_unique_ids = []
batch_split_ids = []


In [4]:
def eval_batch_onebyone(index_list,batch_id):
    print("Thread {} is running".format(batch_id))
    model2message = {
    'cogvideox5b':"12 frames from cogvideox5b,which you need to evaluate \n",
    'kling':"10 frames from kling ,which you need to evaluate\n ", 
    'gen3': "10 frames from gen3 ,which you need to evaluate\n",
    'videocrafter2':"4 frames from videocrafter2,which you need to evaluate",
    'pika':"7 frames from pika ,which you need to evaluate",
    'show1':"8 frames from show1,which you need to evaluate ",
    'lavie':"5 frames from lavie ,which you need to evaluate",
    }
    requests = []
    for i in index_list: 

        examplemodels = [x for x in model2message.keys()]
        eval_res = eval_results[str(i)] 
        frames = videoreader.process_video(data_prepath,human_anno[i]['videos'],2)    
       
        request ={"custom_id": "request-{}".format(i), 
                "method": "POST", 
                "url": "/v1/chat/completions",
                "body": {"model": agent1.model_name,
                            "messages": [],
                            "temperature": 0}}



        prompten = human_anno[i]['prompt_en']
        # question = human_anno[i]['question_en']
        # subject = human_anno[i]['subject_en']
        # scene = human_anno[i]['scene_en']
        # objet = human_anno[i]['object']
        messages=[
        {
        "role": "system", "content":
        agent1.system_message_image,
        }
        ,
        {
        "role": "user", "content": [
        "According to  your **Goals**in system meassage, there are examples and evaluation results from video generation models.\n",
        *[item for examplemodel in examplemodels for item in [
            "This example is from model '{}' \n".format(examplemodel),
            {"type": "image_url", "image_url": {"url": f'data:image/jpg;base64,{frames[examplemodel][0]}', "detail": "low"}}
        ]],                 
        "The evaluation results of these examples are as follows:\n",
        *map(lambda x: f"model: {x}, evaluation result: {eval_res[x]}\n", examplemodels),
        
        "Assuming there are videos you think should score 'x',provide your analysis and explanation in the output format as follows:\n",
        *map(lambda x: f"- {x}: should be scored x,because...\n", examplemodels),

            ],
            }
        ]
        request['body']['messages'] = messages
        requests.append(request)

    with open(os.path.join(batch_stpath,"requests_{}_batch_{}.jsonl".format(dimension,batch_id)), "w") as f:
        for entry in requests:
            json_line = json.dumps(entry)
            f.write(json_line + '\n')
    
    batch_input_file = client.files.create(
             file=open(os.path.join(batch_stpath,"requests_{}_batch_{}.jsonl".format(dimension,batch_id)), "rb"),
              purpose="batch"
             )

    batch_input_file_id = batch_input_file.id    

    batch_object = client.batches.create(
            input_file_id=batch_input_file_id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={
            "description": "nightly group1 {} eval job batch {}".format(dimension,batch_id)
            }
                                        )
    
    batch_split_ids.append(batch_id)
    batch_unique_ids.append(batch_object.id)

    print("Thread {} is done".format(batch_id))

In [5]:
# l1 = list(range(1,len(human_anno),3))
# l2 = list(range(2,len(human_anno),3))
l3 = list(range(0,len(human_anno),3))

# l = [int(x) for x in['204', '207', '210', '213', '216', '219', '222', '225', '228', '231']]


ls = l3

In [6]:
import threading

batch_size = 15
batches = [ls[i:i + batch_size] for i in range(0, len(ls), batch_size)]

# with open("./batch_infos/batch_info_{}.json".format(dimension), "r") as f:
#     batch_info = json.load(f)

# batch_split_ids = batch_info['batch_split_ids']
# batches = batch_info['videos_in_batch']

threads = []
for i, batch in enumerate(batches):
    thread = threading.Thread(target=eval_batch_onebyone, args=(batch, i))
    threads.append(thread)
    thread.start()

print("All threads started")
# 等待所有线程完成
for thread in threads:
    thread.join()

#保存batch信息
with open("./batch_infos/batch_info_{}_stablestudy.json".format(dimension), "w") as f:
    json.dump({"batch_unique_ids": batch_unique_ids, "batch_split_ids": batch_split_ids,"videos_in_batch":batches}, f, indent=4)

Thread 0 is running
Thread 1 is running
Thread 2 is running
Thread 3 is running
Thread 4 is running
Thread 5 is running
Thread 6 is running
All threads started
Thread 6 is done
Thread 1 is done
Thread 0 is done
Thread 2 is done
Thread 5 is done
Thread 3 is done
Thread 4 is done


In [14]:
with open("./batch_infos/batch_info_{}_stablestudy.json".format(dimension), "r") as f:
    batch_info = json.load(f)

    
batchids = batch_info["batch_unique_ids"]

critic_evalph = "./GPT4o_eval_results/imaging_quality/imaging_quality_multillmeval_3.json"
with open(critic_evalph,'r') as f:
    critic_eval = json.load(f)
    
for i in ls:
    if str(i) not in critic_eval.keys():
         critic_eval[str(i)] = {}

for id in batchids:
    batch_object = client.batches.retrieve(id)
    print("id:{} status:{} descrepition:{}".format(id,batch_object.status,batch_object.metadata['description']))

    if batch_object.status != "completed":
        print("batch {} is not completed".format(id))
        continue    

    file_response = client.files.content(batch_object.output_file_id)
    for line in file_response.text.splitlines():
        # index = json.loads(line)["custom_id"].split("-")[-2]
        # model = json.loads(line)["custom_id"].split("-")[-1]

        index = json.loads(line)["custom_id"].split("-")[-1]

        eval_res = json.loads(line)["response"]["body"]["choices"][0]["message"]["content"].replace('\n\n','\n')
        
        # critic_eval[index][model] = eval_res
        critic_eval[index] = eval_res
    with open(critic_evalph, "w") as f:
        json.dump(critic_eval, f, indent=4)

    print("batch {} done,end index {}".format(id,index))

id:batch_67349aadcfb48190910bb4eb1b2f9868 status:completed descrepition:nightly group1 imaging_quality eval job batch 6
batch batch_67349aadcfb48190910bb4eb1b2f9868 done,end index 270
id:batch_67349aee1cb88190910d24c19ba7c990 status:completed descrepition:nightly group1 imaging_quality eval job batch 1
batch batch_67349aee1cb88190910d24c19ba7c990 done,end index 87
id:batch_67349afa25dc819088988bbfd626bc35 status:completed descrepition:nightly group1 imaging_quality eval job batch 0
batch batch_67349afa25dc819088988bbfd626bc35 done,end index 42
id:batch_67349afd7abc8190a1a5d3bc8c295046 status:completed descrepition:nightly group1 imaging_quality eval job batch 2
batch batch_67349afd7abc8190a1a5d3bc8c295046 done,end index 132
id:batch_67349afd83ec8190bb36157e91f26161 status:completed descrepition:nightly group1 imaging_quality eval job batch 5
batch batch_67349afd83ec8190bb36157e91f26161 done,end index 267
id:batch_67349afdf59c8190b7b484856cff1a49 status:completed descrepition:nightly gr